In [ ]:
!pip install -q -U kaggle-environments
!pip list | grep kaggle

In [ ]:
import base64
import bz2
import pickle

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

model_path = {
    "a": "../input/hungry-geese-models/first_stage_2462.pth",
    "b": "../input/hungry-geese-models/IMO_pretrain_1st_4_280.pth",
    # "c": "../input/hungry-geese-models/latest.pth",
    # "d": "../input/hungry-geese-models/second_stage_3257.pth",
    # "e": "../input/hungry-geese-models/IMO_pretrain_1st_8_550.pth",
    # "f": "../input/hungry-geese-models/CNN_first_2_2540.pth",
    # "g": "../input/hungry-geese-models/CNN_first_2_2545.pth",
}

PARAM = {}
for key, val in model_path.items():
    weights = torch.load(val)
    PARAM[key] = base64.b64encode(bz2.compress(pickle.dumps(weights)))

In [ ]:
%%writefile submission.py

PARAM = {
    "a": b"aaaaaaaaaa",
    "imo_b": b"bbbbbbbbbb",
    # "2_c": b"cccccccccc",
    # "d": b"dddddddddd",
    # "imo_e": b"eeeeeeeeee",
    # "2_f": b"ffffffffff",
    # "2_g": b"gggggggggg",
}

# This is a lightweight ML agent trained by self-play.
# After sharing this notebook,
# we will add Hungry Geese environment in our HandyRL library.
# https://github.com/DeNA/HandyRL
# We hope you enjoy reinforcement learning!


import base64
import bz2
import math
import pickle
from collections import defaultdict

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# Neural Network for Hungry Geese


class Dense(nn.Module):
    def __init__(self, units0, units1, bnunits=0, bias=True):
        super().__init__()
        if bnunits > 0:
            bias = False
        self.dense = nn.Linear(units0, units1, bias=bias)
        self.bnunits = bnunits
        self.bn = nn.BatchNorm1d(bnunits) if bnunits > 0 else None

    def forward(self, x):
        h = self.dense(x)
        if self.bn is not None:
            size = h.size()
            h = h.view(-1, self.bnunits)
            h = self.bn(h)
            h = h.view(*size)
        return h


class TorusConv2d(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, bn):
        super().__init__()
        self.edge_size = (kernel_size[0] // 2, kernel_size[1] // 2)
        self.conv = nn.Conv2d(input_dim, output_dim, kernel_size=kernel_size)
        self.bn = nn.BatchNorm2d(output_dim) if bn else None

    def forward(self, x):
        h = torch.cat([x[:, :, :, -self.edge_size[1] :], x, x[:, :, :, : self.edge_size[1]]], dim=3)
        h = torch.cat([h[:, :, -self.edge_size[0] :], h, h[:, :, : self.edge_size[0]]], dim=2)
        h = self.conv(h)
        h = self.bn(h) if self.bn is not None else h
        return h


class Conv2d(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, bn):
        super().__init__()
        self.conv = nn.Conv2d(input_dim, output_dim, kernel_size=kernel_size)
        self.bn = nn.BatchNorm2d(output_dim) if bn else None

    def forward(self, x):
        h = self.conv(x)
        h = self.bn(h) if self.bn is not None else h
        return h


class ChannelSELayer(nn.Module):
    def __init__(self, channel, reduction=8):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid(),
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)


# https://github.com/Kaixhin/Rainbow/blob/master/model.py
# Factorised NoisyLinear layer with bias
class NoisyLinear(nn.Module):
    def __init__(self, in_features, out_features, std_init=0.5):
        super(NoisyLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.std_init = std_init
        self.weight_mu = nn.Parameter(torch.empty(out_features, in_features))
        self.weight_sigma = nn.Parameter(torch.empty(out_features, in_features))
        self.register_buffer("weight_epsilon", torch.empty(out_features, in_features))
        self.bias_mu = nn.Parameter(torch.empty(out_features))
        self.bias_sigma = nn.Parameter(torch.empty(out_features))
        self.register_buffer("bias_epsilon", torch.empty(out_features))
        self.reset_parameters()
        self.reset_noise()

    def reset_parameters(self):
        mu_range = 1 / math.sqrt(self.in_features)
        self.weight_mu.data.uniform_(-mu_range, mu_range)
        self.weight_sigma.data.fill_(self.std_init / math.sqrt(self.in_features))
        self.bias_mu.data.uniform_(-mu_range, mu_range)
        self.bias_sigma.data.fill_(self.std_init / math.sqrt(self.out_features))

    def _scale_noise(self, size):
        x = torch.randn(size, device=self.weight_mu.device)
        return x.sign().mul_(x.abs().sqrt_())

    def reset_noise(self):
        epsilon_in = self._scale_noise(self.in_features)
        epsilon_out = self._scale_noise(self.out_features)
        self.weight_epsilon.copy_(epsilon_out.ger(epsilon_in))
        self.bias_epsilon.copy_(epsilon_out)

    def forward(self, input):
        if self.training:
            return F.linear(
                input,
                self.weight_mu + self.weight_sigma * self.weight_epsilon,
                self.bias_mu + self.bias_sigma * self.bias_epsilon,
            )
        else:
            return F.linear(input, self.weight_mu, self.bias_mu)


class GeeseNet(nn.Module):
    def __init__(self):
        super().__init__()
        layers, filters = 12, 32
        self.conv0 = TorusConv2d(17, filters, (3, 3), True)
        self.blocks = nn.ModuleList([TorusConv2d(filters, filters, (3, 3), True) for _ in range(layers)])

        self.conv_p = TorusConv2d(filters, filters, (3, 3), True)
        self.conv_v = TorusConv2d(filters, filters, (3, 3), True)

        self.head_p = nn.Linear(filters, 4, bias=False)
        self.head_v1 = nn.Linear(filters * 2, filters, bias=False)
        self.head_v2 = nn.Linear(filters, 1, bias=False)

    def forward(self, x, _=None):
        h = F.relu_(self.conv0(x))
        for block in self.blocks:
            h = F.relu_(h + block(h))

        h_p = F.relu_(self.conv_p(h))
        h_head_p = (h_p * x[:, :1]).view(h_p.size(0), h_p.size(1), -1).sum(-1)
        p = self.head_p(h_head_p)

        h_v = F.relu_(self.conv_v(h))
        h_head_v = (h_v * x[:, :1]).view(h_v.size(0), h_v.size(1), -1).sum(-1)
        h_avg_v = h_v.view(h_v.size(0), h_v.size(1), -1).mean(-1)

        h_v = F.relu_(self.head_v1(torch.cat([h_head_v, h_avg_v], 1)))
        v = torch.tanh(self.head_v2(h_v))

        return {"policy": p, "value": v, "h_head_p": h_head_p, "h_head_v": h_head_v, "h_avg_v": h_avg_v}


class GeeseNet2(nn.Module):
    def __init__(self):
        super().__init__()
        layers, filters = 6, 40

        self.conv0 = TorusConv2d(17, filters, (3, 3), True)
        self.cnn_blocks = nn.ModuleList([TorusConv2d(filters, filters, (3, 3), True) for _ in range(layers)])
        self.cse_blocks = nn.ModuleList([ChannelSELayer(filters, reduction=4) for _ in range(layers)])

        self.conv_p = TorusConv2d(filters, filters, (3, 3), True)
        self.conv_v = TorusConv2d(filters, filters, (3, 3), True)

        self.head_p = NoisyLinear(filters, 4)
        self.head_v1 = NoisyLinear(77, 16)
        self.head_v2 = NoisyLinear(16, 1)

    def forward(self, x, _=None):
        h = F.relu_(self.conv0(x))
        for cnn, cse in zip(self.cnn_blocks, self.cse_blocks):
            h = cnn(h)
            h = F.relu_(h + cse(h))

        p = F.relu_(self.conv_p(h))
        head = x[:, :1]
        p_head = (p * head).view(h.size(0), h.size(1), -1).sum(-1)
        p = self.head_p(p_head)

        v = F.relu_(self.conv_v(h))
        v_gap = v.view(h.size(0), h.size(1), -1).mean(1)
        v = F.relu_(self.head_v1(v_gap))
        v = torch.tanh(self.head_v2(v))

        return {"policy": p, "value": v}

    def reset_noise(self):
        for name, module in self.named_children():
            if "head" in name:
                module.reset_noise()


class GeeseNetIMO(nn.Module):
    class GeeseBlock(nn.Module):
        def __init__(self, embed_dim, num_heads):
            super().__init__()
            self.attention = nn.MultiheadAttention(embed_dim, num_heads)

        def forward(self, x):
            h, _ = self.attention(x, x, x)
            return h

    class GeeseControll(nn.Module):
        def __init__(self, filters, final_filters):
            super().__init__()
            self.filters = filters
            self.attention = nn.MultiheadAttention(filters, 1)
            self.fc_control = Dense(filters * 3, final_filters, bnunits=final_filters)

        def forward(self, x, e):
            h, _ = self.attention(x, x, x)

            h = torch.cat([x, e, h], dim=2).view(x.size(1), -1)
            h = self.fc_control(h)
            return h

    class GeeseHead(nn.Module):
        def __init__(self, filters):
            super().__init__()
            f = filters // 2
            self.head_p_1 = nn.Linear(filters, f, bias=False)
            self.head_p_2 = nn.Linear(f, 4, bias=False)
            self.head_v_1 = nn.Linear(filters, f, bias=True)
            self.head_v_2 = nn.Linear(f, 1, bias=True)

        def forward(self, x):
            p = self.head_p_1(x)
            p = self.head_p_2(p)
            v = self.head_v_1(x)
            v = torch.tanh(self.head_v_2(v))
            return p, v

    def __init__(self):
        super().__init__()
        blocks = 5
        filters = 64
        final_filters = 128

        self.geese_net = GeeseNet()

        self.blocks = nn.ModuleList([self.GeeseBlock(filters, 8) for _ in range(blocks)])
        self.control = self.GeeseControll(filters, final_filters)
        self.head = self.GeeseHead(final_filters)

    def forward(self, x, _=None):
        x_ = self.geese_net(x)
        e = torch.cat([x_["h_head_p"], x_["h_avg_v"]], 1).view(1, x.size()[0], -1)
        h = e
        for block in self.blocks:
            h = block(h)
        h = self.control(h, e)
        p, v = self.head(h)
        return {"policy": p, "value": v}


# Input for Neural Network


NUM_ROW = 7
NUM_COL = 11
CENTER_ROW = NUM_ROW // 2
CENTER_COL = NUM_COL // 2


def to_offset(x):
    row = CENTER_ROW - x // NUM_COL
    col = CENTER_COL - x % NUM_COL
    return row, col


def to_row(offset, x):
    return (x // NUM_COL + offset) % NUM_ROW


def to_col(offset, x):
    return (x + offset) % NUM_COL


def make_input(obses):
    b = np.zeros((17, 7 * 11), dtype=np.float32)
    obs = obses[-1]

    for p, pos_list in enumerate(obs["geese"]):
        # head position
        for pos in pos_list[:1]:
            b[0 + (p - obs["index"]) % 4, pos] = 1
        # tip position
        for pos in pos_list[-1:]:
            b[4 + (p - obs["index"]) % 4, pos] = 1
        # whole position
        for pos in pos_list:
            b[8 + (p - obs["index"]) % 4, pos] = 1

    # previous head position
    if len(obses) > 1:
        obs_prev = obses[-2]
        for p, pos_list in enumerate(obs_prev["geese"]):
            for pos in pos_list[:1]:
                b[12 + (p - obs["index"]) % 4, pos] = 1

    # food
    for pos in obs["food"]:
        b[16, pos] = 1

    return b.reshape(-1, 7, 11)


def make_input_centering_head_for_rule(obses):
    b = {}
    for i in range(4):
        b[i] = defaultdict(list)
    obs = obses[-1]

    player_goose_head = obs["geese"][obs["index"]][0]
    o_row, o_col = to_offset(player_goose_head)

    for p, geese in enumerate(obs["geese"]):
        # whole position
        for pos in geese:
            b[(p - obs["index"]) % 4]["body"].append((to_row(o_row, pos), to_col(o_col, pos)))

    # previous head position
    if len(obses) > 1:
        obs_prev = obses[-2]
        for p, geese in enumerate(obs_prev["geese"]):
            for pos in geese[:1]:
                b[(p - obs["index"]) % 4]["previous"].append((to_row(o_row, pos), to_col(o_col, pos)))

    # food
    for pos in obs["food"]:
        b[0]["food"].append((to_row(o_row, pos), to_col(o_col, pos)))

    return b


def distance(a, b):
    x = b[0] - a[0]
    y = b[1] - a[1]
    return (x, y), abs(x) + abs(y)


def around(a):
    return [
        ((a[0] - 1) % 7, a[1]),
        ((a[0] + 1) % 7, a[1]),
        (a[0], (a[1] - 1) % 11),
        (a[0], (a[1] + 1) % 11),
    ]


def apply_rule(b, prob):
    """
    player head = (3, 5)
    ["NORTH", "SOUTH", "WEST", "EAST"]
    """
    north = (2, 5)
    south = (4, 5)
    west = (3, 4)
    east = (3, 6)
    neighbor = [north, south, west, east]

    # 隣接している場所に行けないケース
    for i, n in enumerate(neighbor):
        # 自分の直前の場所
        if n in b[0]["previous"]:
            prob[i] = -np.inf

        for p in range(4):
            # ガチョウの体がある場所 (しっぽ除く)
            if n in b[p]["body"][:-1]:
                prob[i] = -np.inf

    north_2step = [(2, 4), (1, 5), (2, 6)]
    south_2step = [(4, 4), (5, 5), (4, 6)]
    west_2step = [(2, 4), (3, 3), (4, 4)]
    east_2step = [(2, 6), (3, 7), (4, 6)]
    two_step = [north_2step, south_2step, west_2step, east_2step]

    # 2step 先のマスがすべて、2step後に埋まっている場合、移動不可とする
    for i, ts in enumerate(two_step):
        death = 0
        for s in ts:
            for p in range(4):
                if s in b[p]["body"][:-2]:
                    death += 1
                    break
            else:
                break
        if death == len(ts):
            prob[i] -= 10_000

    # 次の移動で頭がぶつかる可能性のあるケース
    for p in range(1, 4):
        if b[p]["body"] != []:
            (x, y), d = distance(b[0]["body"][0], b[p]["body"][0])
            if d == 2:
                if x < 0:
                    prob[0] -= 100 if north in b[0]["food"] else 50
                elif x > 0:
                    prob[1] -= 100 if south in b[0]["food"] else 50
                if y < 0:
                    prob[2] -= 100 if west in b[0]["food"] else 50
                elif y > 0:
                    prob[3] -= 100 if east in b[0]["food"] else 50

    # しっぽが伸びる可能性のあるケース
    for i, n in enumerate(neighbor):
        for p in range(1, 4):
            if (
                b[p]["body"] != []
                and n == b[p]["body"][-1]
                and any(food in around(b[p]["body"][0]) for food in b[0]["food"])
            ):
                prob[i] -= 100

    return prob


# Load PyTorch Model


model = {}
for key, param in PARAM.items():
    state_dict = pickle.loads(bz2.decompress(base64.b64decode(param)))
    if "imo_" in key:
        model[key] = GeeseNetIMO()
    elif "2_" in key:
        model[key] = GeeseNet2()
    else:
        model[key] = GeeseNet()
    model[key].load_state_dict(state_dict)
    model[key].eval()


# Main Function of Agent

obses = []


def agent(obs, _):
    obses.append(obs)
    x = make_input(obses)
    y = make_input_centering_head_for_rule(obses)

    preds = np.zeros((len(PARAM), 4), dtype=np.float32)
    for i, key in enumerate(PARAM.keys()):
        with torch.no_grad():
            xt = torch.from_numpy(x).unsqueeze(0)
            o = model[key](xt)
        p = o["policy"].squeeze(0).detach().numpy()
        preds[i] = p

    inf = np.mean(preds, axis=0)
    inf = apply_rule(y, inf)

    actions = ["NORTH", "SOUTH", "WEST", "EAST"]
    return actions[np.argmax(inf)]

In [ ]:
# Read in the submission file
with open(
    "submission.py",
) as file:
    filedata = file.read()

# Replace the target string
for key, val in PARAM.items():
    filedata = filedata.replace(key * 10, val.decode("utf-8"))

# Write the file out again
with open("submission.py", "w") as file:
    file.write(filedata)

In [ ]:
from kaggle_environments import make

env = make("hungry_geese", debug=True)

env.reset()
env.run(["submission.py", "submission.py", "submission.py", "submission.py"])
env.render(mode="ipython", width=800, height=700)